In [115]:
import marketdata.ext
import marketdata
import pandas as pa
from IPython.display import HTML
import pybacktest as pbt
import datetime as dt
from IPython.display import HTML

In [2]:
#data loading
price_b = marketdata.ext.load_metotron('SPY')

In [175]:
#data preparing
start = dt.datetime(2000, 1, 1)
ohlc = pa.DataFrame(price_b, columns=('O', 'H', 'L', 'C')).ix[start:]

In [188]:
#strategy code
O, H, L, C = ohlc.O, ohlc.H, ohlc.L, ohlc.C
ATR = pa.rolling_mean(H-L, 25)
LLV = pa.rolling_min(L, 10)
IBS = (C-L)/(H-L)
UpBand = L + ATR
ChUp = (C > UpBand)
ChIBS = (IBS > 0.85)
ChRet = ((O/C.shift(1) - 1) > 0.001)
short =  ChUp & ChIBS & ChRet.shift(-1)#code have a hack here (for simplicity of research) and cannot be placed in prod without mode
cover = short.shift(1)# and here. also, need to modify to exit on close
buy = sell = 0
buyprice = shortprice = O
sellprice = coverprice = C
res = dict(Buy=buy, Sell=sell, Short=short, Cover=cover)
bct = pbt.Backtest(dict(ohlc=ohlc, buy=res['Buy'], sell=res['Sell'],short=res['Short'], cover=res['Cover']), name = 'SPY diverge')
perf = bct.report
#bct.trades.head(20)
#bct.plot_equity()

In [158]:
#checking table
test = ohlc
test['ATR'] = pa.rolling_mean(test.H-test.L, 25)
test['LLC'] = pa.rolling_min(test.L, 10)
test['IBS'] = (test.C - test.L) / (test.H - test.L)
test['UpBand'] = test.L + test.ATR
test['C_UpB'] = (test.C > test.UpBand)
test['CheckIBS'] = (test.IBS > 0.85)
test['Return'] = (test.O/test.C.shift(1) -1) > 0.001
test['Short'] = (test.C_UpB) & (test.CheckIBS) & (test.Return.shift(-1))
test['Cover'] = test.Short.shift(1)
test = test.dropna(how = 'any')

In [163]:
#H = lambda df: HTML(test.head(100).to_html())
#tt = H(test)

In [164]:
start = test.index.searchsorted(dt.datetime(2000, 3, 14))
end = test.index.searchsorted(dt.datetime(2000, 3, 18))
df1 = test.ix[start:end]